In [71]:
# A dictionary of movie critics and their ratings of a small
# set of movies
# this nested dic contain first, name and movies as key  and value
# then inside it contains movie and rating as key and values

critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 3.5},
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
'The Night Listener': 4.5, 'Superman Returns': 4.0,
'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 2.0},
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [72]:
time_critics = {'time': {'superman': 1, 'batman':2, 'star war': 3},'tuna':{'superman': 5, 'batman':4, 'star war': 3}}

Euclidean Distance Score

distance btwn 2 point 1/(distance+1) so the range will be 0-1 and for perfect distance (same point) it will be 1
since we allready +1 to distance we wont get DNE as answer

In [73]:
from math import sqrt

# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
    
    # Get the list of shared_items
    si={}
    
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
            
    # if they have no ratings in common, return 0
    if len(si)==0: return 0
    
    # Add up the squares of all the differences
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2)
        for item in prefs[person1] if item in prefs[person2]])
    return 1/(1+sum_of_squares)


In [74]:
sim_distance(critics,'Lisa Rose','Gene Seymour')

0.14814814814814814

Pearson correlation coefficient

set two keys (in this case persons) as the x,y axis then locate common movie that they have on the graph 
the algorithm is just to simply draw the straight (best-fit) line on chart, and it
comes as close to all the items on the chart as possible

range of pearson correlation is -1 to 1 (1 = perfect)

In [75]:
# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
    # Get the list of mutually rated items
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item]=1
            
    # Find the number of elements
    n=len(si)
    
    # if they are no ratings in common, return 0
    if n==0: return 0
    
    # Add up all the preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
    
    # Sum up the squares
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    
    # Sum up the products
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    
    # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0
    
    r=num/den
    
    return r

In [76]:
sim_pearson(critics,'Lisa Rose','Gene Seymour')

0.39605901719066977

In [77]:
sim_pearson(time_critics,'time','tuna')

-1.0

the experiment from time_critics data set show that for Pearson correlation coefficient the rating of two person
do not need to be exactly the same to get the perfect scare output (1)
However, if the rating ration keep on being the same (as one inclease the score another inclease so they think in the same direction but they may give different number of score but same ratio), it will give out perfect score since the best-fit line pass throught all data

On the other hand, -1 mean that Best fit line has slope negative so it imply that as another one think one movie is good, another will think it is bad --> we can say that their idea is opposite

but if the data does not close on both end (-1 or 1) we cant really extract any patter out of the data


======================================================================================

function below here is used to calculate the top list of people who have same interest as the input person

optional !
the function will return result of top-n people (default is 5)
the algorithm to calculate can be Pearson (tell relationship of like and dislike, but the score do not need to be the same)
or
Distance (tell how close your score and those people score are)

In [78]:
# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.
def topMatches(prefs,person,n =5,similarity = sim_pearson):
    scores=[(similarity(prefs,person,other),other)
        for other in prefs if other!=person]
    # Sort the list so the highest scores appear at the top
    scores.sort( )
    scores.reverse( )
    return scores[0:n]

In [79]:
topMatches(critics,'Toby',n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

Recomended system (this function is well explained) 
but you should see a chart in the book page 15.

In [80]:
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
        # don't compare me to myself
        if other==person: continue
            
        sim=similarity(prefs,person,other)
        
        # ignore scores of zero or lower
        if sim<=0: continue
        for item in prefs[other]:
            
            # only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:
                
                # Similarity * Score
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                
                # Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim
                
    # Create the normalized list
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]
    
    # Return the sorted list
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [81]:
getRecommendations(critics,'Toby',similarity=sim_distance)

[(3.5002478401415877, 'The Night Listener'),
 (2.7561242939959363, 'Lady in the Water'),
 (2.461988486074374, 'Just My Luck')]

In [82]:
getRecommendations(critics,'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

Introduce to item base filtering 

from earlier code we focus on the similarity of people then we use that to find the people who have close taste as us
we can further use that info of similarity in taste to recommend movies that we havent watched base on taste of ppl 
that are similar

next we will transpose the matrix to see what can we achive

In [83]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            
            # Flip item and person
            result[item][person]=prefs[person][item]
    return result

In [84]:
movie = transformPrefs(critics)

so as we transpose the old critics dict to movie dict 
the topMatches function return the list of the movie that are similar to the given movie [top 3] which means a person gives these to movie a close rating
if more people give the same rate to these movie it will get closer to one else if more ppl give rating different the to go to -1



In [85]:
topMatches(movie,'Superman Returns',n=3)

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane')]

so now the recommendation function will take the movie and then seek for other similar movies then it can recomment who (that havnt seen) should watch this input movie

In [86]:
getRecommendations(movie,'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

In [87]:
def loadMovieLens(path='C:/Users/tappasarn/Downloads/ml-1m'):
    # Get movie titles
    movies={}
    for line in open(path+'/movies.dat'):
        (id,title)=line.split('::')[0:2]
        movies[id]=title
        
    # Load data
    prefs={}
    for line in open(path+'/ratings.dat'):
        (user,movieid,rating,ts)=line.split('::')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs

In [90]:
movielens_data = loadMovieLens()
getRecommendations(movielens_data,'87')

[(5.0, 'Time of the Gypsies (Dom za vesanje) (1989)'),
 (5.0, 'Tigrero: A Film That Was Never Made (1994)'),
 (5.0, 'Schlafes Bruder (Brother of Sleep) (1995)'),
 (5.0, 'Return with Honor (1998)'),
 (5.0, 'Lured (1947)'),
 (5.0, 'Identification of a Woman (Identificazione di una donna) (1982)'),
 (5.0, 'I Am Cuba (Soy Cuba/Ya Kuba) (1964)'),
 (5.0, 'Hour of the Pig, The (1993)'),
 (5.0, 'Gay Deceivers, The (1969)'),
 (5.0, 'Gate of Heavenly Peace, The (1995)'),
 (5.0, 'Foreign Student (1994)'),
 (5.0, 'Dingo (1992)'),
 (5.0, 'Dangerous Game (1993)'),
 (5.0, 'Callej\xf3n de los milagros, El (1995)'),
 (5.0, 'Bittersweet Motel (2000)'),
 (4.820460101722989, 'Apple, The (Sib) (1998)'),
 (4.738956184936386, 'Lamerica (1994)'),
 (4.681816541467396, 'Bells, The (1926)'),
 (4.664958072522234, 'Hurricane Streets (1998)'),
 (4.650741840804559, 'Sanjuro (1962)'),
 (4.649974172600346, 'On the Ropes (1999)'),
 (4.636825408739507, 'Shawshank Redemption, The (1994)'),
 (4.627888709544556, 'For All M